In [ ]:
from common import *
import mahotas
from scipy import spatial
import scipy.stats

import numpy.random

In [ ]:
db = load_db(download_tar("http://vision.lems.brown.edu/sites/default/files/216db.tar.gz"))

In [ ]:
def query(image, descriptor, distance, top_n = 5, db=db):

    C = np.stack([descriptor(im) for im in db.image])

    desc = descriptor(image)
    dists = distance( desc, C )

    
    return db.iloc[np.argsort(dists)].head(n=top_n)


def test_queries(descriptor, distance, top_n=5, z_score_desc=False, i=None, transform=lambda im: im):

    # Get descriptors for all images
    # but apply `transform` first (a way to introduce noise)

    _db = db.copy()

    _db.image  = _db.image.apply(transform)
    
    _db["desc"]= [descriptor(im) for im in _db.image]

    if z_score_desc:
        D = np.stack(_db.desc.to_numpy())
        # Round to 10 digits to avoid unwanted variance from numerical precision
        D = scipy.stats.zscore(np.round(D,10), axis=0)
        # Fillna to make sure
        D = np.nan_to_num(D, nan=0)
        _db["desc"] = list(D)

    queries = []
    results=[]
    for clas in _db.clas.unique():

        if i is None:
            row = _db.query(f"clas=='{clas}'").sample(1).iloc[0]
        else:
            row = _db.query(f"clas=='{clas}'").iloc[i]


        C = np.stack(_db.desc.to_numpy())

        # desc = row.desc
        dists = distance( row.desc, C)

        queries.append(pd.DataFrame(data=[row]))#[["clas","image"]])
        results.append(_db.iloc[np.argsort(dists)].head(n=top_n))#[["clas","image"]])
    
    return queries, results

def query_acc(query, result):
    return np.sum(result.clas == query.clas.iloc[0])/len(result)

def query_accuracies(queries, results):
    res = pd.DataFrame()
    for q, r in zip(queries,results):
        res[f"{q.clas.iloc[0]}"]=pd.Series(query_acc(q,r))
    res.index.rename("Accuracy", inplace=True)
    return res

def rand_rotate(im, scale_range=(0.85,1.15), rotation_range=(-180,180)):
    h,w = im.shape
    s_low, s_high = scale_range
    r_low, r_high = rotation_range
    M = cv2.getRotationMatrix2D(
        # Keep center
        center=(w//2,h//2),
        # Use random ints for rotation (1 deg resolution is ok)
        angle=np.random.randint(low=r_low,high=r_high),
        # Use random float for scale
        scale=np.random.rand()*(s_high-s_low)+s_low,
    )
    return cv2.warpAffine(im, M, dsize=im.shape[::-1])

In [ ]:
def show_query_results(queries, results):
    qDf = {}
    rDf = pd.DataFrame()
    for i, (q, r) in enumerate(zip(queries, results)):
        qDf[f"{q.clas.iloc[0]}"] = q.image.iloc[0]
        rDf[f"{q.clas.iloc[0]}"] = r.image.reset_index(drop=True)

    print("queries:")
    displayImages(pd.DataFrame(data=[qDf]))
    print("results:")
    displayImages(rDf)

def show_query_accuracy(queries, results):
    print("Accuracies:")
    acc = query_accuracies(queries,results)
    display(acc)
    print(f"Average along classes: {acc.transpose().mean().iloc[0]:.2f}")

In [ ]:
def hu_moments(im):
    im = 255*(im!=0).astype(np.uint8)
    h=cv2.moments(im)

    return cv2.HuMoments(h).flatten()

def log_transform(vec):
    return -1* np.copysign(1.0, vec) * np.log10(abs(vec))

def hu_moments_log(im):
    hu = hu_moments(im)
    return log_transform(hu)


def l1Dist(vec, dataset):
    return np.linalg.norm( vec.reshape(1,-1) - dataset , axis=-1, ord=1)

def l2Dist(vec, dataset):
    return np.linalg.norm( vec.reshape(1,-1) - dataset , axis=-1, ord=2)

In [ ]:
def regularize(image):

    A = np.vstack(np.where(image!=0)[::-1])
    center = np.mean(A,axis=-1)
    A = A.T - center
    u, s, vt = np.linalg.svd(A)

    if np.linalg.det(vt) < 0:
        vt[0,:] = vt[0,:]*-1

    r= vt
    c = center
    dsize = (250,250)

    center_d = (dsize[0]/2, dsize[1]/2)
    W = np.zeros((2,3))
    W[:2,:2] = r 
    W[:2,2] = -r@c  + center_d

    im2 = cv2.warpAffine(image, W, dsize=dsize)
    return im2

In [ ]:
displayDf(query(db.iloc[6].image, descriptor=hu_moments_log, distance=l2Dist, top_n=5))

In [ ]:
queries, results = test_queries(hu_moments, l1Dist, top_n=5, i=0, z_score_desc=True)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
queries, results = test_queries(hu_moments_log, l1Dist, top_n=5, z_score_desc=False, i=0)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
def zernike(image, r=60, degree=8):
    return  mahotas.features.zernike_moments(image, r, degree=degree)
def cosine_distance(vec, dataset):
    return [spatial.distance.cosine(vec, data) for data in dataset]

In [ ]:

displayDf(query(db.image.iloc[5], zernike, l2Dist, top_n=5) )

In [ ]:
queries, results = test_queries(zernike, l2Dist, top_n=5, z_score_desc=True, i=3)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
queries, results = test_queries(zernike, l2Dist, top_n=5, z_score_desc=True, i=3, transform=rand_rotate)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
def regularize_scale(im, object_size=115, output_size=115):
    im = im

    cmax = get_largest_contour(im)

    x,y,w,h = cv2.boundingRect(cmax)

    # plt.imshow(im)
    # plt.plot([c[0,0] for c in cmax], [c[0,1] for c in cmax], color="r")
    # plt.figure()
    # plt.imshow(im[y:y+h,x:x+w])
    # m = np.zeros((2,3))
    # m[0,0]=1
    # m[1,1]=1
    # m[0,2] = dsize//2 - w//2
    # m[1,2] = dsize//2 - h//2

    im2 = im[y:y+h,x:x+w]

    factor = object_size/max(h,w)
    im2 = cv2.resize(im2, dsize=None, fx=factor,fy=factor)

    h,w = im2.shape
    border_h = output_size-h 
    border_w = output_size-w

    top = border_h//2
    bottom = border_h-top

    left = border_w//2
    right = border_w-left

    im2 = cv2.copyMakeBorder(im2, top, bottom, left, right, cv2.BORDER_CONSTANT)

    return im2

In [ ]:
from functools import partial

# db2 = db.copy()
# db2.image = db2.image.apply(regularize)
# db2.image = db2.image.apply(partial(regularize_scale, object_size=115, output_size=200))
# for image in db2.query("clas=='turtle'").image:
#     cmax = get_largest_contour(255-image)
#     plt.plot([c[0,0] for c in cmax], [c[0,1] for c in cmax], color="r")

# for image in db2.query("clas=='elephant'").image:
#     cmax = get_largest_contour(255-image)
#     plt.plot([c[0,0] for c in cmax], [c[0,1] for c in cmax], color="g")

In [ ]:
db.image = db.image.apply(regularize)
db.image = db.image.apply(partial(regularize_scale, object_size=115, output_size=150))

In [ ]:
def mask(image):
    return (image==0).flatten()

def hamming(desc, C):
    return np.sum(desc ^ C, axis=1)

In [ ]:
queries, results = test_queries(mask, hamming, top_n=5, i=0)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
queries, results = test_queries(mask, hamming, top_n=5, i=0, transform=rand_rotate)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
import skimage

In [ ]:
im = db.query("clas=='ray'").image.iloc[7]


p = (200-155)//2

params = dict(
    orientations=5,
    pixels_per_cell=(13,13),
    cells_per_block=(1,1), 
    visualize=True
    )

im_f = cv2.blur(im,(5,5))
fd, hog_image = skimage.feature.hog(im[p:-p,p:-p], **params)

print(fd.shape)
plt.imshow(im)
plt.figure()
plt.imshow(hog_image)

def hog_desc(im):
    im_f = cv2.blur(im,(5,5))
    fd, hog_image = skimage.feature.hog(im_f[p:-p,p:-p], **params)
    return fd

In [ ]:
queries, results = test_queries(hog_desc, cosine_distance, top_n=5)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
import torch
import clip

model, preprocess = clip.load("ViT-B/32", device="cpu")

In [ ]:
plt.imshow(db.image.iloc[0])
plt.show()

In [ ]:
im = db.query("clas=='bird'").image.iloc[0]

In [ ]:
from PIL import Image

image = Image.fromarray(np.uint8(im))
device = "cpu"
image = preprocess(image).unsqueeze(0).to(device)
text = clip.tokenize("a " + db.clas.unique()).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

In [ ]:
i = np.argmax(probs)
db.clas.unique()[i]

In [ ]:
def clip_embedding(im):
    image = Image.fromarray(np.uint8(im))
    device = "cpu"
    image = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)

    return image_features.numpy().flatten()

In [ ]:
queries, results = test_queries(clip_embedding, l2Dist, top_n=5)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
queries, results = test_queries(clip_embedding, l2Dist, top_n=5)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

def vgg_embedding(im):
    image = Image.fromarray(np.uint8(im))
    device = "cpu"
    image = preprocess(image).unsqueeze(0).to(device)
    return vgg16.avgpool(image).numpy().ravel()

In [ ]:
queries, results = test_queries(vgg_embedding, l2Dist, top_n=5)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
queries, results = test_queries(vgg_embedding, cosine_distance, top_n=5)
show_query_results(queries, results)
show_query_accuracy(queries, results)

In [ ]:
im = db.image.iloc[1]